In [ ]:
#训练中量化测试
#很奇怪，训练中量化能量化个啥东西出来？
import torch
from torch.ao.quantization import QConfigMapping#torch1.13才有
import torch.quantization.quantize_fx as quantize_fx
import copy

import sys
sys.path.append("../") 
from Mymodels import Vit


# from Mymodels import Test_Net


In [ ]:
In_Channels=3
Embed_Dim=384
Picture_Size=224
Patch_Size=16
Num_Class=3
Num_Heads=6
Encoder_Layers=6
Onnx_Export_Path='Exported_Fx_Onnx.onnx'
model_fp=Vit(In_Channels=In_Channels,Out_Channels=Embed_Dim,Picture_Size=Picture_Size,Patch_Size=Patch_Size
,Num_Class=Num_Class,Num_Heads=Num_Heads,Encoder_Layers=Encoder_Layers)

In [24]:
input_fp32=torch.rand(1,In_Channels,Picture_Size,Picture_Size)
model_to_quantize = copy.deepcopy(model_fp)
qconfig_mapping = QConfigMapping().set_global(torch.quantization.get_default_qat_qconfig('qnnpack'))
model_to_quantize.train()
# prepare
model_prepared = quantize_fx.prepare_qat_fx(model_to_quantize, qconfig_mapping, input_fp32)
# training loop (not shown)
# quantize
model_quantized = quantize_fx.convert_fx(model_prepared)
print(model_quantized)
model_to_quantize = copy.deepcopy(model_fp)
model_fused = quantize_fx.fuse_fx(model_to_quantize)

c:\Users\25073\.conda\envs\YoloV5\lib\site-packages\torch\ao\quantization\utils.py:287: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


AttributeError: 'FusedMovingAvgObsFakeQuantize' object has no attribute 'activation_post_process'

In [ ]:
#训练


In [22]:
torch.onnx.export(model_quantized,               # model being run
                input_fp32,                         # model input 
                'Exportedonnx.onnx',   # where to save the model (can be a file or file-like object)                  
                opset_version=16,           # the ONNX version to export the model to                  
                input_names = ['input'],   # the model's input names
                output_names = ['output'],  # the model's output names
                export_params=True
                )
#
# fusion
#


RuntimeError: 
Module 'FusedMovingAvgObsFakeQuantize' has no attribute 'activation_post_process' :
  File "c:\Users\25073\.conda\envs\YoloV5\lib\site-packages\torch\ao\quantization\fake_quantize.py", line 289
    @torch.jit.export
    def calculate_qparams(self) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.activation_post_process.calculate_qparams()
               ~~~~~~~~~~~~~~~~~~~~~~~~~~~~ <--- HERE


In [ ]:
a=100
export_path='Exported_Onnx'+str(a)+'.onnx'
export_path